---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    mydf = pd.read_csv('university_towns.txt', sep='\n', header=None, names=['RegionName']) #read the file, use sep for the 'enter new line', name the column 
    mydf['State'] = np.where(mydf['RegionName'].str.contains('edit'),mydf['RegionName'],np.NaN) #create another column for state that has edit, fill nan for one that doesn't
    mydf['State'].fillna(method='ffill', inplace=True) #forward fill the nan with state name
    mydf['State']= mydf['State'].str.replace(r'\[.*\]','') #remove the []
    mydf['RegionName']= mydf['RegionName'].str.replace(r' \(.*','') #remove the ()
    mydf['RegionName']= mydf['RegionName'].str.replace(r'\[.*','') #remove the []
    mydf = mydf[mydf['RegionName']!=mydf['State']] #remove the duplicate in column
    mydf = mydf[['State','RegionName']] #reorder the columns
    return mydf
get_list_of_university_towns()

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
...,...,...
561,Wisconsin,River Falls
562,Wisconsin,Stevens Point
563,Wisconsin,Waukesha
564,Wisconsin,Whitewater


In [4]:
gdp = pd.read_excel('gdplev.xls', skiprows=5) #read file, drop rows that not needed

In [5]:
gdp = gdp.drop(gdp.columns[[0,1,2,3,5,7]], axis=1) #drop columns and rows that are not needed
gdp = gdp.drop(gdp.index[[0,1]])

In [6]:
gdp.columns.values[0:2] = ['Time', 'GDP in billions'] #change name of columns

In [7]:
gdp['GDP difference'] = gdp['GDP in billions'].diff() #calc differences among rows

In [8]:
gdp = gdp[212:] #only look at 2008q1 onward

In [9]:
gdp.reset_index(drop=True, inplace=True)  

In [10]:
def get_recession_start():
    mylist = []
    for i in range(len(gdp)):
        if (gdp['GDP difference'][i] < 0 and gdp['GDP difference'][i+1] <0):
            mylist.append(gdp['Time'][i])
    return mylist[0]

In [11]:
get_recession_start()

'2008q3'

In [12]:
# another way to do it
# gdp_dec = gdp.where(gdp['GDP difference']<0)
# gdp_dec = gdp_dec.dropna()
# gdp_dec['Index'] = gdp_dec.index
# gdp_dec['Index Difference'] = gdp_dec['Index'].diff()
# gdp_dec
# gdp.iloc[gdp_dec['Index Difference'].idxmin()-1]['Time']

In [13]:
indexstart = get_recession_start() #find the index where the recession start
indexend = gdp[gdp['Time'] == indexstart].index.tolist()[0] #you need the [0] at the end because the tolist return a list. 
gdp2 = gdp.iloc[indexend:] #if you don't grab the element in here, you can't slide it
gdp2.reset_index(drop=True, inplace=True)  
gdp2.head(6)

,Time,GDP in billions,GDP difference
0,2008q3,14891.6,-71.8
1,2008q4,14577.0,-314.6
2,2009q1,14375.0,-202.0
3,2009q2,14355.6,-19.4
4,2009q3,14402.5,46.9
5,2009q4,14541.9,139.4


In [14]:
def get_recession_end():
    mylist2 = []
    for i in range(len(gdp2)):
        if (gdp2['GDP difference'][i] > 0 and gdp['GDP difference'][i+1] >0):
            mylist2.append(gdp2['Time'][i+1])
    return mylist2[0]

In [15]:
get_recession_end()

'2009q4'

In [16]:
def get_recession_bottom():
    start = get_recession_start()
    end = get_recession_end()
    indexstart = gdp[gdp['Time']== start].index.tolist()[0]
    indexend = gdp[gdp['Time']== end].index.tolist()[0]
    gdp3 = gdp.iloc[indexstart:indexend]
    return gdp3['Time'][gdp3['GDP in billions'].idxmin]
get_recession_bottom()

'2009q2'

In [37]:
housing = pd.read_csv('City_Zhvi_AllHomes.csv')
housing = housing.drop(housing.loc[:, '1996-04':'1999-12' ], axis =1) #drop columns not needed
housing = housing.drop(housing.columns[[0,3,4,5]], axis =1) #drop columns not needed
housing['State'] = housing['State'].map(states) #map from 2 letters state name to full state name

#switch 2 columns
columnsName = list(housing.columns)
S, R = columnsName.index('State'), columnsName.index('RegionName')
columnsName[S], columnsName[R] = columnsName[R],columnsName[S]
housing = housing[columnsName]

In [38]:
def convert_quarter(x):
    year = x.split('-')[0]
    quarter = int((int(x.split('-')[1])-1)/3+1)
    newx = year + 'q' + str(quarter)
    return newx

In [20]:
# col1 = housing.columns[2:].tolist()
# col1 = [convert_quarter(x) for x in col1]
# housing.rename(columns=dict(zip(housing.columns[2:], col1)),inplace=True)

In [39]:
housing = housing.set_index(['State', 'RegionName']).sort_index()

2000-01   2000-02   2000-03   2000-04   2000-05  \
State   RegionName                                                      
Alabama Adamsville    68700.0   69100.0   69300.0   69200.0   69100.0   
        Alabaster    121600.0  122300.0  122500.0  122800.0  123200.0   
        Albertville   74300.0   74100.0   73500.0   72700.0   72500.0   
        Arab          84500.0   83900.0   82900.0   81900.0   81400.0   
        Ardmore           NaN       NaN       NaN       NaN       NaN   
...                       ...       ...       ...       ...       ...   
Wyoming Burns        100900.0  101300.0  102400.0  103600.0  104400.0   
        Casper        88400.0   89300.0   90000.0   90100.0   89600.0   
        Cheyenne     116000.0  116700.0  117900.0  119200.0  120100.0   
        Evansville   128500.0  128100.0  127500.0  127700.0  128600.0   
        Pine Bluffs   93900.0   93400.0   93900.0   94900.0   95300.0   

                      2000-06   2000-07   2000-08   2000-09   2000-10  ...  \
State   RegionName                                                     ...   
Alabama Adamsville    69200.0   69500.0   69700.0   70200.0   71100.0  ...   
        Alabaster    123200.0  123100.0  123100.0  123300.0  123500.0  ...   
        Albertville   72600.0   72500.0   72800.0   73200.0   73500.0  ...   
        Arab          81400.0   81200.0   81200.0   81600.0   82400.0  ...   
        Ardmore           NaN       NaN       NaN       NaN       NaN  ...   
...                       ...       ...       ...       ...       ...  ...   
Wyoming Burns        105700.0  107000.0  108200.0  109900.0  111700.0  ...   
        Casper        89100.0   89000.0   89500.0   90700.0   92100.0  ...   
        Cheyenne     120800.0  121000.0  121400.0  122200.0  123000.0  ...   
        Evansville   130000.0  130900.0  130600.0  131000.0  132300.0  ...   
        Pine Bluffs   95000.0   94100.0   94200.0   95600.0   97200.0  ...   

                     2015-11  2015-12  2016-01  2016-02  2016-03  2016-04  \
State   RegionName                                                          
Alabama Adamsville     73900    73500    73600    74100    74800    75200   
        Alabaster     151800   152400   152800   153400   154200   154900   
        Albertville    91600    91500    91400    92000    92600    92600   
        Arab          112500   112100   111000   109800   109300   109500   
        Ardmore       144200   143100   142200   142500   143000   143200   
...                      ...      ...      ...      ...      ...      ...   
Wyoming Burns         164100   164500   166800   168500   169500   171100   
        Casper        182300   182600   184000   184500   184500   184900   
        Cheyenne      190500   191900   193800   194700   194800   195500   
        Evansville    303900   306600   308200   309700   310400   310200   
        Pine Bluffs   169600   169300   168400   166300   165800   166600   

                     2016-05  2016-06  2016-07  2016-08  
State   RegionName                                       
Alabama Adamsville     74900    74700    74700    74700  
        Alabaster     155200   155200   155600   156100  
        Albertville    92400    92400    92300    92100  
        Arab          110100   110700   111600   112400  
        Ardmore       143400   143100   142300   141600  
...                      ...      ...      ...      ...  
Wyoming Burns         172100   171600   170700   170300  
        Casper        185400   185200   184600   184100  
        Cheyenne      196400   197600   198800   199400  
        Evansville    308800   308600   308800   309300  
        Pine Bluffs   164500   160300   158000   157300  

[10730 rows x 200 columns]

In [41]:
housing = housing.groupby(convert_quarter, axis = 1).mean()

2000q1         2000q2         2000q3  \
State   RegionName                                                 
Alabama Adamsville    69033.333333   69166.666667   69800.000000   
        Alabaster    122133.333333  123066.666667  123166.666667   
        Albertville   73966.666667   72600.000000   72833.333333   
        Arab          83766.666667   81566.666667   81333.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        101533.333333  104566.666667  108366.666667   
        Casper        89233.333333   89600.000000   89733.333333   
        Cheyenne     116866.666667  120033.333333  121533.333333   
        Evansville   128033.333333  128766.666667  130833.333333   
        Pine Bluffs   93733.333333   95066.666667   94633.333333   

                            2000q4         2001q1         2001q2  \
State   RegionName                                                 
Alabama Adamsville    71966.666667   73466.666667   74000.000000   
        Alabaster    123700.000000  123233.333333  125133.333333   
        Albertville   74200.000000   75900.000000   76000.000000   
        Arab          82966.666667   84200.000000   84533.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        113000.000000  115833.333333  117200.000000   
        Casper        93166.666667   95500.000000   97633.333333   
        Cheyenne     123633.333333  125533.333333  126300.000000   
        Evansville   132066.666667  130566.666667  131433.333333   
        Pine Bluffs   98066.666667  103233.333333  104600.000000   

                            2001q3         2001q4         2002q1  \
State   RegionName                                                 
Alabama Adamsville    73333.333333   73100.000000   73333.333333   
        Alabaster    127766.666667  127200.000000  127300.000000   
        Albertville   72066.666667   73566.666667   76533.333333   
        Arab          81666.666667   83900.000000   87266.666667   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        117800.000000  117633.333333  117333.333333   
        Casper        99433.333333  100633.333333  101733.333333   
        Cheyenne     126466.666667  128133.333333  128466.666667   
        Evansville   132400.000000  133466.666667  133300.000000   
        Pine Bluffs  106500.000000  104066.666667  102233.333333   

                            2002q2  ...         2014q2         2014q3  \
State   RegionName                  ...                                 
Alabama Adamsville    73133.333333  ...   77066.666667   75966.666667   
        Alabaster    128000.000000  ...  147133.333333  147633.333333   
        Albertville   76366.666667  ...   84033.333333   84766.666667   
        Arab          87700.000000  ...  113366.666667  111700.000000   
        Ardmore                NaN  ...  140533.333333  139566.666667   
...                            ...  ...            ...            ...   
Wyoming Burns        117233.333333  ...  168866.666667  161933.333333   
        Casper       101533.333333  ...  175766.666667  177300.000000   
        Cheyenne     129633.333333  ...  177466.666667  176733.333333   
        Evansville   131066.666667  ...  296733.333333  305666.666667   
        Pine Bluffs  103566.666667  ...  148666.666667  154366.666667   

                            2014q4         2015q1         2015q2  \
State   RegionName                                                 
Alabama Adamsville    71900.000000   71666.666667   73033.333333   
        Alabaster    148700.000000  148900.000000  149566.666667   
        Albertville   86800.000000   88466.666667   89500.000000   
        Arab         111600.000000  110166.666667  109433.333333   
        Ardmore

In [43]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return housing
convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State   RegionName                                                 
Alabama Adamsville    69033.333333   69166.666667   69800.000000   
        Alabaster    122133.333333  123066.666667  123166.666667   
        Albertville   73966.666667   72600.000000   72833.333333   
        Arab          83766.666667   81566.666667   81333.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        101533.333333  104566.666667  108366.666667   
        Casper        89233.333333   89600.000000   89733.333333   
        Cheyenne     116866.666667  120033.333333  121533.333333   
        Evansville   128033.333333  128766.666667  130833.333333   
        Pine Bluffs   93733.333333   95066.666667   94633.333333   

                            2000q4         2001q1         2001q2  \
State   RegionName                                                 
Alabama Adamsville    71966.666667   73466.666667   74000.000000   
        Alabaster    123700.000000  123233.333333  125133.333333   
        Albertville   74200.000000   75900.000000   76000.000000   
        Arab          82966.666667   84200.000000   84533.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        113000.000000  115833.333333  117200.000000   
        Casper        93166.666667   95500.000000   97633.333333   
        Cheyenne     123633.333333  125533.333333  126300.000000   
        Evansville   132066.666667  130566.666667  131433.333333   
        Pine Bluffs   98066.666667  103233.333333  104600.000000   

                            2001q3         2001q4         2002q1  \
State   RegionName                                                 
Alabama Adamsville    73333.333333   73100.000000   73333.333333   
        Alabaster    127766.666667  127200.000000  127300.000000   
        Albertville   72066.666667   73566.666667   76533.333333   
        Arab          81666.666667   83900.000000   87266.666667   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        117800.000000  117633.333333  117333.333333   
        Casper        99433.333333  100633.333333  101733.333333   
        Cheyenne     126466.666667  128133.333333  128466.666667   
        Evansville   132400.000000  133466.666667  133300.000000   
        Pine Bluffs  106500.000000  104066.666667  102233.333333   

                            2002q2  ...         2014q2         2014q3  \
State   RegionName                  ...                                 
Alabama Adamsville    73133.333333  ...   77066.666667   75966.666667   
        Alabaster    128000.000000  ...  147133.333333  147633.333333   
        Albertville   76366.666667  ...   84033.333333   84766.666667   
        Arab          87700.000000  ...  113366.666667  111700.000000   
        Ardmore                NaN  ...  140533.333333  139566.666667   
...                            ...  ...            ...            ...   
Wyoming Burns        117233.333333  ...  168866.666667  161933.333333   
        Casper       101533.333333  ...  175766.666667  177300.000000   
        Cheyenne     129633.333333  ...  177466.666667  176733.333333   
        Evansville   131066.666667  ...  296733.333333  305666.666667   
        Pine Bluffs  103566.666667  ...  148666.666667  154366.666667   

                            2014q4         2015q1         2015q2  \
State   RegionName                                                 
Alabama Adamsville    71900.000000   71666.666667   73033.333333   
        Alabaster    148700.000000  148900.000000  149566.666667   
        Albertville   86800.000000   88466.666667   89500.000000   
        Arab         111600.000000  110166.666667  109433.333333   
        Ardmore

In [47]:
df = convert_housing_data_to_quarters()
uni = get_list_of_university_towns().set_index(['State', 'RegionName'])
uni

Empty DataFrame
Columns: []
Index: [(Alabama, Auburn), (Alabama, Florence), (Alabama, Jacksonville), (Alabama, Livingston), (Alabama, Montevallo), (Alabama, Troy), (Alabama, Tuscaloosa), (Alabama, Tuskegee), (Alaska, Fairbanks), (Arizona, Flagstaff), (Arizona, Tempe), (Arizona, Tucson), (Arkansas, Arkadelphia), (Arkansas, Conway), (Arkansas, Fayetteville), (Arkansas, Jonesboro), (Arkansas, Magnolia), (Arkansas, Monticello), (Arkansas, Russellville), (Arkansas, Searcy), (California, Angwin), (California, Arcata), (California, Berkeley), (California, Chico), (California, Claremont), (California, Cotati), (California, Davis), (California, Irvine), (California, Isla Vista), (California, University Park, Los Angeles), (California, Merced), (California, Orange), (California, Palo Alto), (California, Pomona), (California, Redlands), (California, Riverside), (California, Sacramento), (California, University District, San Bernardino), (California, San Diego), (California, San Luis Obispo), (California, Santa Barbara), (California, Santa Cruz), (California, Turlock), (California, Westwood, Los Angeles), (California, Whittier), (Colorado, Alamosa), (Colorado, Boulder), (Colorado, Durango), (Colorado, Fort Collins), (Colorado, Golden), (Colorado, Grand Junction), (Colorado, Greeley), (Colorado, Gunnison), (Colorado, Pueblo, Colorado), (Connecticut, Fairfield), (Connecticut, Middletown), (Connecticut, New Britain), (Connecticut, New Haven), (Connecticut, New London), (Connecticut, Storrs), (Connecticut, Willimantic), (Delaware, Dover), (Delaware, Newark), (Florida, Ave Maria), (Florida, Boca Raton), (Florida, Coral Gables), (Florida, DeLand), (Florida, Estero), (Florida, Gainesville), (Florida, Orlando), (Florida, Sarasota), (Florida, St. Augustine), (Florida, St. Leo), (Florida, Tallahassee), (Florida, Tampa), (Georgia, Albany), (Georgia, Athens), (Georgia, Atlanta), (Georgia, Carrollton), (Georgia, Demorest), (Georgia, Fort Valley), (Georgia, Kennesaw), (Georgia, Milledgeville), (Georgia, Mount Vernon), (Georgia, Oxford), (Georgia, Rome), (Georgia, Savannah), (Georgia, Statesboro), (Georgia, Valdosta), (Georgia, Waleska), (Georgia, Young Harris), (Hawaii, Manoa), (Idaho, Moscow), (Idaho, Pocatello), (Idaho, Rexburg), (Illinois, Carbondale), (Illinois, Champaign–Urbana), (Illinois, Charleston), (Illinois, DeKalb), (Illinois, Edwardsville), ...]

[517 rows x 0 columns]

In [46]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    df = convert_housing_data_to_quarters()
    
    # Start position is the quarter BEFORE the recession starts!
    before_rec = (df.columns.get_loc(get_recession_start())-1)
    rec_bottom = df.columns.get_loc(get_recession_bottom())
    
    uni = get_list_of_university_towns().set_index(['State', 'RegionName'])
    
    # Turn the divided values into a DataFrame!
    df = np.divide(df.iloc[:,before_rec],df.iloc[:,rec_bottom]).to_frame().dropna()
    
    # Merge university and GDP data.
    uni_df = df.merge(uni, right_index=True, left_index=True, how='inner')
    
    # Drop the indices of uni towns to get data only for non uni towns.
    nonuni_df = df.drop(uni_df.index)
    
    # A t-test is commonly used to determine whether the mean of a population significantly
    # differs from a specific value (called the hypothesized mean) or from the mean of another population.
    p_value = ttest_ind(uni_df.values, nonuni_df.values).pvalue
    
    if p_value < 0.01:
        different=True
    else:
        different=False
        
    # Better depending on which one is LOWER! Remember prices go up during a recession so lower is better.
    if uni_df.mean().values < nonuni_df.mean().values:
        better='university town'
    else:
        better='non-university town'

    return (different, p_value[0], better)
    
run_ttest()

(True, 0.002724063704761164, 'university town')